# Conjugate Gradient and Preconditioned Conjugate Gradient methods


### Error Analysis

Stationary methods

* Jacobi method

$$x^{(k+1)}_i = \frac{1}{A_{i,i}}\left(b_i-\sum_{j\neq i}A_{i,j}x^{(k)}_j\right)$$

In vectorized form
$${\bf x}^{(k+1)} = D^{-1}({\bf b}-(A-D){\bf x}^{(k)})$$

* Gauss Seidel method

$$x^{(k+1)}_i = \frac{1}{A_{i,i}}\left(b_i-\sum_{j<i}A_{i,j}x^{(k+1)}_j-\sum_{j>i}A_{i,j}x^{(k)}_j\right)$$

In vectorized form
$${\bf x}^{(k+1)} = L^{-1}({\bf b}-U{\bf x}^{(k)})$$

* Over Successive Relaxation 

$$x^{(k+1)}_i = \omega\frac{1}{A_{i,i}}\left(b_i-\sum_{j<i}A_{i,j}x^{(k+1)}_j-\sum_{j>i}A_{i,j}x^{(k)}_j\right) + (1-\omega)x^{(k)}_i$$

In vectorized form
$${\bf x}^{(k+1)} = \omega L^{-1}({\bf b}-U{\bf x}^{(k)}) + (1-\omega){\bf x}^{(k)}=\omega{\bf x}^{(k+1)}_{GS}+(1-\omega){\bf x}^{(k)}$$

These methods can all be summarized as: (*stationary method*)

$${\bf x}^{(k+1)} = {\bf x}^{(k)}+M^{-1}({\bf b}-A{\bf x}^{(k)})$$

* Jacobi:
\begin{align}
{\bf x}^{(k+1)} &= D^{-1}({\bf b}-(A-D){\bf x}^{(k)})\\
{\bf x}^{(k+1)} &= D^{-1}({\bf b}-A{\bf x}^{(k)}+D{\bf x}^{(k)})\\
{\bf x}^{(k+1)} &= {\bf x}^{(k)}+D^{-1}({\bf b}-A{\bf x}^{(k)})
\end{align}

* Gauss-Seidel:
\begin{align}
{\bf x}^{(k+1)} &= L^{-1}({\bf b}-U{\bf x}^{(k)})\\
{\bf x}^{(k+1)} &= L^{-1}({\bf b}-(U+L-L){\bf x}^{(k)})\\
{\bf x}^{(k+1)} &= L^{-1}({\bf b}-(A-L){\bf x}^{(k)})\\
{\bf x}^{(k+1)} &= {\bf x}^{(k)}+L^{-1}({\bf b}-A{\bf x}^{(k)})
\end{align}

<div style="background-color:rgba(256, 256, 0, 0.1); padding:10px 0;font-family:monospace;">
    <b>Theorem: Stationary Method Convergence.</b><br><hr>
    For the linear problem $A{\bf x}={\bf b}$, consider the iterative method $${\bf x}^{(k+1)}={\bf x}^{(k)}+M^{-1}{\bf r}^{(k)},$$ where ${\bf r}^{(k)}={\bf b}-A{\bf x}^{(k)}$ is the residue at k-th step, and define the <b>iteration matrix</b> $T=I-M^{-1}A$. Then the method converges if and only if the spectral radius of the iteration matrix satisfies $$\rho(T)<1$$ The smaller $\rho(T)$, the faster the convergence. 
</div>

#### Proof of the theorem:
Let $x^*$ be the exact solution, so $A{\bf x}^*={\bf b}$, and ${\bf r}^{(k)} = {\bf b}-A{\bf x}^{(k)}=A({\bf x}^*-{\bf x}^{(k)})$
\begin{align}
{\bf x}^{(k+1)}&={\bf x}^{(k)}+M^{-1}{\bf r}^{(k)}\\
{\bf x}^{(k+1)}-{\bf x}^*&={\bf x}^{(k)}-{\bf x}^*+M^{-1}{\bf r}^{(k)}\\
{\bf e}^{(k+1)} &= {\bf e}^{(k)}-M^{-1}A{\bf e}^{(k)}\\
{\bf e}^{(k+1)} &= (I-M^{-1}A){\bf e}^{(k)}\\
{\bf e}^{(k+1)} &= T{\bf e}^{(k)}\\
\end{align}
From there, we get
\begin{align}
\|{\bf e}^{(k+1)}\|_2&\leq \|T\|_2\|{\bf e}^{(k)}\|_2\\
&\leq \rho(T)\|{\bf e}^{(k)}\|_2\\
&\leq \rho(T)^2\|{\bf e}^{(k-1)}\|_2\\
&\leq \rho(T)^{k+1}\|{\bf e}^{(0)}\|_2
\end{align}

So, we know that $\|{\bf e}^{(k)}\|_2=C\rho(T)^{k}$<br>
If you take $\log_{10}$ from both sides, you can see that 
$$\log_{10}(\|{\bf e}^{(k)}\|_2) = k \log_{10}(\rho(T))+\log_{10}C$$

### Conjugate Gradient Method

Conjugate gradient method is derived from **steepest descent method**. So we need to first talk about steepest descent method. <br>
Solving $A{\bf x}={\bf b}$ is equivalent to minimizing the following functional:
$$\phi({\bf x})=\frac{1}{2}{\bf x}^TA{\bf x}-{\bf b}^T{\bf x}$$
Gradient descent iteration:
$${\bf x}^{(k+1)}={\bf x}^{(k)}+\alpha_k{\bf r}^{(k)}$$
Which leads to minimizing:
$$\frac{1}{2}({\bf x}^{(k)}+\alpha_k{\bf r}^{(k)})^TA({\bf x}^{(k)}+\alpha_k{\bf r}^{(k)})-{\bf b}^T({\bf x}^{(k)}+\alpha_k{\bf r}^{(k)})$$
Taking derivative with respect to $\alpha_k$, we get 
$$\alpha_k{\bf r}^{(k)T}A{\bf r}^{(k)}+{\bf r}^{(k)T}A{\bf x}^{(k)}-{\bf r}^{(k)T}{\bf b}=0$$
This leads us to 
$$\alpha_k=\dfrac{{\bf r}^{(k)T}{\bf r}^{(k)}}{{\bf r}^{(k)T}A{\bf r}^{(k)}}$$

The **Conjugatet Gradient Method** takes the search direction ${\bf p}^{(k)}$ no longer the same as the residual vector ${\bf r}^{(k)}$, but the search direction ${\bf p}^{(k)}$ needs to be perpendicular to all the previous search directions ${\bf p}^{(j)}$ for $j<k$. 

<div style="background-color:rgba(0, 256, 0, 0.1); padding:10px 0;font-family:monospace;">
    <b>Algorithm: Conjugate Gradient Method.</b><br><hr>
    Given an initial guess ${\bf x}_0$ and a tolerance $\epsilon$, set at first ${\bf r}_0={\bf b}-A{\bf x}_0$, $\delta_0=\langle {\bf r}_0, {\bf r}_0\rangle$, ${\bf b}_{\delta} = \langle {\bf b}_0, {\bf b}_0\rangle$, $k=0$, and ${\bf p}_0={\bf r}_0$. Then:<br>
    &nbsp;&nbsp;&nbsp;&nbsp; while $\delta_k>\epsilon^2{\bf b}_{\delta}$:<br>
    &nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp; ${\bf s}_k=A{\bf p}_k$<br>
    &nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp; $\alpha_k = \dfrac{\delta_k}{\langle {\bf p}_k, {\bf s}_k\rangle}$<br>
    &nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp; ${\bf x}_{k+1} = {\bf x}_{k}+\alpha_k{\bf p}_k$<br>
    &nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp; ${\bf r}_{k+1} = {\bf r}_{k}-\alpha_k{\bf s}_k$<br>
    &nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp; $\delta_{k+1} = \langle {\bf r}_{k+1}, {\bf r}_{k+1}\rangle$<br>
    &nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp; ${\bf p}_{k+1} = {\bf r}_{k+1}+\dfrac{\delta_{k+1}}{\delta_{k}} {\bf p}_{k}$<br>
    &nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp; $k = k+1$
</div>

In [19]:
def CG(A, b, tol=1e-6):
    x = np.zeros_like(b)
    r = b - A@x
    delta = r@r
    delta_b = b@b
    k = 0
    p = r
    while delta>tol*tol*delta_b:
        s = A@p
        alpha = delta/(p@s)
        x += alpha*p
        r -= alpha*s
        delta_old = delta
        delta = r@r
        p = r+(delta/delta_old)*p
        k += 1
    return x, k

In [5]:
import numpy as np
from scipy.sparse import dok_matrix
def ind(i,j,n):
    return i+j*(n+1)
def formingSparseLinearSystem(n, g):
    h = 1/n
    A = dok_matrix(((n+1)**2,(n+1)**2), dtype = np.float64)
    b = np.zeros((n+1)**2)
    for i in range(n+1):
        for j in range(n+1):
            if 0<i<n and 0<j<n:
                A[ind(i,j,n),ind(i,j,n)] = 4
                A[ind(i,j,n),ind(i-1,j,n)] = -1
                A[ind(i,j,n),ind(i+1,j,n)] = -1
                A[ind(i,j,n),ind(i,j-1,n)] = -1
                A[ind(i,j,n),ind(i,j+1,n)] = -1
                b[ind(i,j,n)] = g(j*h,i*h)*h*h
            else:
                A[ind(i,j,n),ind(i,j,n)] = 1
                b[ind(i,j,n)] = 0
    A = A.tocsr()
    return A, b

In [25]:
g = lambda x,y: x*(1-x)+y*(2-y)
n = 30
As, bs = formingSparseLinearSystem(n, g)

In [26]:
def CG(A, b, tol=1e-6):
    x = np.zeros_like(b)
    r = b - A@x
    delta = r@r
    delta_b = b@b
    k = 0
    p = r
    while delta>tol*tol*delta_b:
        s = A@p
        alpha = delta/(p@s)
        x += alpha*p
        r -= alpha*s
        delta_old = delta
        delta = r@r
        p = r+(delta/delta_old)*p
        k += 1
    return x, k

In [27]:
x, k = CG(As, bs,tol=1e-6)

In [28]:
k

857

In [37]:
def cb(k):
    k = k+1
#    print(k)
    return k

k = 0

from scipy.sparse.linalg import cg
x = cg(As, bs, callback = cb)
